In [6]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
from keras.models import Sequential
from tensorflow import keras
from scipy import signal
import os

In [7]:
# assign directory
directory = "drive/MyDrive/MINT_ORIENTATION_DATA"
 
# iterate over files in
# that directory
dataset=[]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        ds = pd.read_csv(f)
        i=ds.loc[ds[' EXG Channel 0'] == 0].index
        if(len(i)==1):
          ds=ds.tail(-(i[0]+1))
        dataset.append(ds)

In [9]:
dataset[6]

,Sample Index,EXG Channel 0,EXG Channel 1,EXG Channel 2,EXG Channel 3,EXG Channel 4,EXG Channel 5,EXG Channel 6,EXG Channel 7,Accel Channel 0,...,Other.3,Other.4,Other.5,Other.6,Analog Channel 0,Analog Channel 1,Analog Channel 2,Timestamp,Other.7,Timestamp (Formatted)
123,1.0,-68981.819628,-100394.469547,55315.560736,10554.918415,86464.549478,69794.730222,28551.291353,19407.706786,0.042,...,29.0,192.0,2.0,208.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:15.656
124,2.0,-68948.135549,-100363.110049,56530.221585,11007.094205,86361.686750,70215.278294,29178.436599,19894.572484,0.042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:15.656
125,3.0,-68896.413612,-100308.728255,54295.985913,9776.965949,86013.558330,69184.862874,27677.829883,18247.807711,0.042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:15.656
126,4.0,-68937.183194,-100344.289880,53619.711533,9588.853668,86168.165346,68994.984805,27377.288327,18083.701203,0.042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:15.656
127,5.0,-68983.183084,-100390.043901,54849.906844,10304.802394,86399.371791,69580.667565,28243.440776,19095.721137,0.042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:15.656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3961,255.0,-69167.093237,-100763.474496,53239.240138,10396.109271,86203.592861,69534.444157,28089.303147,18147.403675,0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:31.230
3962,0.0,-69207.862819,-100803.417063,54372.428879,11090.354453,86483.347295,70106.939388,28913.255502,19122.207954,0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:31.230
3963,1.0,-69205.448831,-100805.518127,56349.708897,12003.602028,86569.669732,70916.564276,30087.191473,20226.227668,0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:31.230
3964,2.0,-69135.666685,-100739.848702,54546.727782,10912.926306,86125.831142,70027.076605,28809.252835,18734.002856,0.026,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666555e+09,0.0,2022-10-23 13:04:31.230


In [14]:
for data in dataset:
  data.drop(data.iloc[:, 9:25], axis=1, inplace=True)
dataset[6].shape

(3843, 9)

In [16]:
i =0;
for data in dataset:
  if data.shape[0] < 3584:
    i=i+1
i

0

In [18]:
len(dataset)

31

In [22]:
data_train=dataset[0:20]
data_test=dataset[20:32]

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler =MinMaxScaler()
i=0;
for data in dataset[0:20]:
  data_train[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1
i=0
for data in dataset[20:31]:
  data_test[i] = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)
  i=i+1

In [24]:
import tqdm #0 is R
n=256*7
feature_names=list(data_train[0].drop('Sample Index',axis=1).columns)
X=[]
Y=[]
indexes=[]
for data in data_train:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(2)):
    X.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
print(X[39].shape)
print(Y[39])

(1792, 8)
1


In [31]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import adam_v2
n_steps=256*7
n_features=8
model=Sequential()
model.add(LSTM(100, activation='relu', return_sequences=False, input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(Dense(1))
opt = adam_v2.Adam(lr=0.0005, decay=1e-6)

model.compile(optimizer="adam", loss='mse', metrics=['mape'])

In [32]:
history=model.fit(np.asarray(X),np.asarray(Y),batch_size=20,epochs=15)

Epoch 1/15
2/2 [==============================] - 3s 755ms/step - loss: 0.3576 - mape: 51216848.0000
Epoch 2/15
2/2 [==============================] - 2s 753ms/step - loss: 0.2912 - mape: 88979576.0000
Epoch 3/15
2/2 [==============================] - 2s 751ms/step - loss: 0.2010 - mape: 123862896.0000
Epoch 4/15
2/2 [==============================] - 2s 742ms/step - loss: 0.1773 - mape: 180545664.0000
Epoch 5/15
2/2 [==============================] - 1s 725ms/step - loss: 0.1497 - mape: 218882944.0000
Epoch 6/15
2/2 [==============================] - 2s 767ms/step - loss: 0.1332 - mape: 241778784.0000
Epoch 7/15
2/2 [==============================] - 1s 736ms/step - loss: 0.1280 - mape: 237088560.0000
Epoch 8/15
2/2 [==============================] - 2s 791ms/step - loss: 0.0964 - mape: 163559696.0000
Epoch 9/15
2/2 [==============================] - 1s 737ms/step - loss: 0.0837 - mape: 114840088.0000
Epoch 10/15
2/2 [==============================] - 2s 774ms/step - loss: 0.0663 - ma

In [33]:
n=256*7
feature_names=list(data_test[0].drop('Sample Index',axis=1).columns)
X_test=[]
Y_test=[]
indexes=[]
for data in data_test:
  norm_data_x =data[feature_names]
  for i in tqdm.tqdm_notebook(range(2)):
    X_test.append(norm_data_x.iloc[i*n:i*n+n].  values)
    Y_test.append(i)
    indexes.append(data.index[i+n-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
predictions = model.predict(np.asarray(X_test))
predictions = pd.DataFrame(predictions).rename(columns={0: 'prediction'})
actual=pd.DataFrame(Y_test).rename(columns={0: 'actual'})
final=pd.concat([predictions,actual],axis=1)
final['mae']=abs(final['prediction']-final['actual'])
final

1/1 [==============================] - 0s 423ms/step


,prediction,actual,mae
0,0.050943,0,0.050943
1,0.913726,1,0.086274
2,0.104888,0,0.104888
3,0.660970,1,0.339030
4,0.084239,0,0.084239
5,0.600704,1,0.399296
6,0.114024,0,0.114024
7,0.710458,1,0.289542
8,0.070513,0,0.070513
9,0.639163,1,0.360837


In [36]:
import pickle
filename = 'finalized_orientation_model.sav'
pickle.dump(model, open(filename, 'wb'))